In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.features import Choropleth, GeoJsonTooltip

# Load dataset
iowa = pd.read_csv('data_2021_to_2024.csv')  # Ensure the dataset is in the correct path

# Feature selection: Drop unnecessary columns
columns_to_drop = [
    'invoice/item_number', 'store_number', 'store_name', 'address',
    'city', 'zip_code', 'store_location', 'county',
    'category', 'category_name', 'vendor_number', 'vendor_name',
    'item_number', 'item_description', 'pack', 'bottle_volume_ml',
    'state_bottle_cost', 'state_bottle_retail', 'bottles_sold',
    'sale_dollars', 'volume_sold_gallons',
    'longitude', 'latitude', 'year', 'month', 'day_of_week',
    'day_of_month'
]

iowa = iowa.drop(columns=columns_to_drop, errors='ignore')  # Use errors='ignore' to avoid errors if columns are missing

# Clean dataset
iowa_promotion = iowa.drop_duplicates()
iowa_promotion_cleaned = iowa_promotion.dropna()

# Check number of unique counties
num_counties = iowa_promotion_cleaned["county"].nunique()
print(f"Number of counties: {num_counties}")

# Analysis for a specific category: "SPICED RUM"
spiced_rum_sales = iowa_promotion_cleaned[iowa_promotion_cleaned["category_name"] == "SPICED RUM"]

# Group by county and calculate average sales
county_avg_sales = spiced_rum_sales.groupby("county")["sale_dollars"].mean().reset_index()

# Sort in descending order and select the top 10 counties
top_10_county_avg_sales = county_avg_sales.sort_values(by="sale_dollars", ascending=False).head(10)
print(top_10_county_avg_sales)

# Generate map with Folium
# Define map center (approximate center of Iowa)
map_center = [41.878, -93.097]  # Iowa central coordinates
store_map = folium.Map(location=map_center, zoom_start=7)

# Add store locations as circle markers
for _, row in top_10_county_avg_sales.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,  # Fixed radius for visibility
        color="blue",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
        popup=folium.Popup(f"County: {row['county']}<br>Avg Sales: ${row['sale_dollars']:.2f}", max_width=300),
    ).add_to(store_map)

# Save the map
store_map.save("top_10_counties_sales_map.html")
print("Map saved as top_10_counties_sales_map.html")
